# 数据的加载

### 从UCI官方加载数据集

In [4]:
from ucimlrepo import fetch_ucirepo

# fetch dataset 
uci_dataset = fetch_ucirepo(id=45)

# data (as pandas dataframes) 
X = uci_dataset.data.features
y = uci_dataset.data.targets

# variable information 
print(uci_dataset.variables)

KeyboardInterrupt: 

### 从UCI加载数据，转化格式，转换为数字形式，并保存为mat数据 
X为特征数据：num_instances,features
Y为lable：(num_instances,1) 是一个列向量

In [9]:

import numpy as np
from sklearn.preprocessing import LabelEncoder
from ucimlrepo import fetch_ucirepo
from scipy.io import savemat

# 数据集
# 76 Nursery 
# 30 Contraceptive Method Choice
# 146 Satellite
# 33 Dermatology
# 23 Chess
uci_dataset = fetch_ucirepo(id=23)
# 1. 特征和标签分离
X = uci_dataset.data.features.values
X_columns = uci_dataset.data.features.columns.tolist()
y = uci_dataset.data.targets.values

# 2. 特征编码
label_encoder = LabelEncoder()
X_encoded = np.copy(X)

# 将特征数据每一列都进行编码
for i in range(X.shape[1]):
    X_encoded[:, i] = label_encoder.fit_transform(X_encoded[:, i])
y_encoded = label_encoder.fit_transform(y[:, 0])

# 转成int格式
X_encoded = X_encoded.astype(int)
y_encoded = y_encoded.astype(int)
# 保存为 .mat 文件
data_dict = {'X': X_encoded, 'Y': y_encoded.reshape(-1, 1)}  # 以字典形式存储
savemat('Chess2.mat', data_dict)
# 输出保存成功
print("数据保存成功！")

数据保存成功！
